In [ ]:
# testing sliding window
from streamsight.splits import SlidingWindowSetting
from streamsight.datasets import AmazonMusicDataset

dataset = AmazonMusicDataset()
data = dataset.load()

setting_sliding = SlidingWindowSetting(
    1106851200,
    # 1398556800,
    window_size=60 * 60 * 24 * 600, # 600 days
    # delta_out= 60 * 60 * 24 * 600
)
setting_sliding.split(data)

In [ ]:
from streamsight.algorithms import ItemKNNIncremental
from streamsight.algorithms.itemknn_rolling import ItemKNNRolling
from streamsight.algorithms.itemknn_static import ItemKNNStatic
from streamsight.metrics.precision import PrecisionK
from streamsight.metrics.recall import RecallK
from streamsight.pipelines.pipeline import MetricAccumulator


acc = MetricAccumulator()
algo1 = ItemKNNIncremental()
algo2 = ItemKNNRolling()
algo3 = ItemKNNStatic()
algo_list = [algo1, algo2, algo3]

results = []
background_data = setting_sliding.background_data

for algo in algo_list:
    algo.fit(background_data)

for i in range(3):
    unlabeled_data = setting_sliding.next_unlabeled_data()
    ground_truth_data = setting_sliding.next_ground_truth_data()
    current_timestamp = setting_sliding.next_data_timestamp_limit()
    
    for algo in algo_list:
        X_pred = algo.predict(unlabeled_data)
        metric1 = PrecisionK(10,current_timestamp)
        metric1.calculate(ground_truth_data.binary_values, X_pred)
        acc.add(metric1, algo.identifier)
        
        metric2 = RecallK(10,current_timestamp)
        metric2.calculate(ground_truth_data.binary_values, X_pred)
        acc.add(metric2, algo.identifier)

        # Release ground truth to model
        algo.fit(ground_truth_data)

In [ ]:
acc.df_by_algo_time()